## **Project Introduction**

---
# Biomimetic Fish Simulation — Robotics Inspired by the Mandarinfish
A Python project simulating the locomotion and goal-oriented behavior of a Mandarinfish (Synchiropus splendidus) — built for applications to robotics master's programs.

---

## General Overview
This project models the undulatory swimming motion and foraging behavior of a single Mandarinfish using Python.
Inspired by principles of natural aquatic locomotion and bio-inspired robotics, the simulation reproduces the fish's characteristic wave-like movement and its behavioral sequence: departing from its coral shelter, foraging for food, and returning home.

---

## Author

- Ibtissem Mokrani
- Contact: ibtissemokrani@gmail.com
- University: Sorbonne Université & Trinity College Dublin (Erasmus)

In [ ]:
# Scientific and numerical libraries
import numpy as np

# Visualization/plotting
import matplotlib.pyplot as plt
from matplotlib import animation

# Other
from IPython.display import HTML
import pandas as pd
import matplotlib.colors as mcolors

!pip install pillow
from google.colab import files

In this section, we simulate the natural undulatory swimming motion of the Mandarinfish.

By generating a sinusoidal wave along the fish's backbone, we can reproduce the fundamental mechanics of real fish locomotion.

These parameters (amplitude, frequency, wavelength) are based on biological measurements and adjusted for coding simplicity.

In [ ]:
# Physical parameters for Mandarinfish

body_length = 6.0         # cm, average for Synchiropus splendidus
n_segments = 20           # number of articulated joints along the body
amplitude = 0.4           # cm, undulatory amplitude
frequency = 2.0           # Hz, tail-beat per second (realistic range 1.5–3)
wavelength = body_length  # cm, typical fish: 1–1.5x body length
speed = 2.0               # cm/s, average for steady cruising

In [ ]:
# Sinusoidal wave model

# Positions along the body (from head to tail), spaced equally
x_segments = np.linspace(0, body_length, n_segments)

# Function that calculate the y deviation for each x and t
def body_wave(x,t):
  A = amplitude
  k = 2 * np.pi / wavelength # wave number
  omega = 2 * np.pi * frequency # angular frequency
  phi = 2 * np.pi / n_segments  # radians, wave phase step between segments
  return A * np.sin(k * x - omega * t + phi * np.arange(len(x))) #with np.arrange(len(x)) for the phase shift


# At t=0:
y_segments_0 = body_wave(x_segments, t=0)
y_segments_0

In [ ]:
# Segment visualisation at t=0s

plt.figure(figsize=(8, 3))
plt.plot(x_segments, y_segments_0, '-o', label='Mandarinfish backbone')
plt.title("Mandarinfish backbone at t=0 s")
plt.xlabel("Length along body (cm)")
plt.ylabel("Lateral displacement (cm)")
plt.legend()
plt.axis('equal')
plt.grid(True, linestyle='--')
plt.tight_layout()

In [ ]:
# Segment visualization at t=0.1s

y_segments_01 = body_wave(x_segments, t=0.1)

plt.figure(figsize=(8, 3))
plt.plot(x_segments, y_segments_01, '-o', label='Mandarinfish backbone')
plt.title("Mandarinfish backbone at t=0.1 s")
plt.xlabel("Length along body (cm)")
plt.ylabel("Lateral displacement (cm)")
plt.legend()
plt.axis('equal')
plt.grid(True, linestyle='--')
plt.tight_layout()
plt.show()

In [ ]:
# Comparison of the Mandarinfish backbone position at t=0, 0.1, 0.2s

y_segments_02 = body_wave(x_segments, t=0.2)


plt.figure(figsize=(8, 3))
plt.plot(x_segments, y_segments_0, '-o', label='Mandarinfish backbone at t=0s')
plt.plot(x_segments, y_segments_01, '-o', label='Mandarinfish backbone at t=0.1s')
plt.plot(x_segments, y_segments_02, '-o', label='Mandarinfish backbone at t=0.2s')
plt.title("Mandarinfish backbone")
plt.xlabel("Length along body (cm)")
plt.ylabel("Lateral displacement (cm)")
plt.legend()
plt.axis('equal')
plt.grid(True, linestyle='--')
plt.tight_layout()
plt.show()



This plot shows the backbone of the Mandarinfish at three different time points (t = 0, 0.1, and 0.2 s). The wave along the body segments represents the instantaneous undulatory shape driving its swimming. The phase shift between frames illustrates the continuous traveling wave propelling the fish forward.

In [ ]:
# Animation of the backbone position through time

# Initial Plot

fig, ax = plt.subplots(figsize=(8,3))
[line] = ax.plot([], [], '-o') # Trailing comma: line is a tuple for FuncAnimation compatibility
ax.set_xlim(0, body_length)
ax.set_ylim(-1.5*amplitude, 1.5*amplitude)
ax.set_xlabel("Length along body (cm)")
ax.set_ylabel("Lateral displacement (cm)")
ax.set_title("Mandarinfish backbone: undulatory swimming")
plt.tight_layout()

def init():
    line.set_data([], [])
    return [line]

def animate(frame):
    t = frame * 0.02
    y = body_wave(x_segments, t)
    line.set_data(x_segments, y)
    return [line]

n_frames = 200 # Number of frames in the animation
anim = animation.FuncAnimation(fig, animate, init_func=init, frames=n_frames, interval=20, blit=True)
plt.close(fig)

# Visualization
HTML(anim.to_jshtml())

In [ ]:
# Download the GIF of the visualization
anim.save('mandarinfish_backbone_swimming.gif', writer='pillow', fps=30)

files.download('mandarinfish_backbone_swimming.gif')

Here, we move from simple wave simulation to purposeful displacement: modeling the way a Mandarinfish travels between its home territory and a food source.

This mimics real foraging behavior found in coral reef environments, and sets the stage for exploring navigation strategies.

In [ ]:
# Trajectory home-food-home
start_pos = np.array([0.0, 0.0])      # Starting position center
home_pos = np.array([0.0, 0.0])       # “Home” location (same as starting position)
food_pos = np.array([19.0, 9.0])     # Food target location

# Cut out the path
n_frames = 200
half_frames = n_frames // 2

# Direction
direction = (food_pos - home_pos) / np.linalg.norm(food_pos - home_pos)  # Unit vector
step_size = np.linalg.norm(food_pos - home_pos) / half_frames # Step size per frame

In [ ]:
# Main simulation figure
fig, ax = plt.subplots(figsize=(8, 3))
[line] = ax.plot([], [], '-o')
[head_marker] = ax.plot([], [], 'o', color='#FFB347', markersize=12, label="Head") # Mandarinfish's head visualization
ax.set_xlim(-2, 25)
ax.set_ylim(-2, 10)
ax.scatter([home_pos[0]], [home_pos[1]], c="g", label="Home", s=50)
ax.scatter([food_pos[0]], [food_pos[1]], c="r", label="Food", s=50)
ax.legend()
plt.title("mandarinfish_backbone_home-food")
plt.tight_layout()

def get_backbone_points(head_pos, angle, t):
    # Aligned segments
    x_rel = np.linspace(0, body_length, n_segments) #head at 0 / tail at 6
    y_lateral = body_wave(x_rel, t)
    x_body = head_pos[0] + x_rel * np.cos(angle) + y_lateral * np.sin(angle)
    y_body = head_pos[1] + x_rel * np.sin(angle) - y_lateral * np.cos(angle)
    return x_body, y_body

def animate(frame):
    # # Decide swimming direction: outbound (home -> food) or return (food -> home)
    if frame < half_frames:
        # Outbound
        move = frame / half_frames
        head = home_pos + (food_pos - home_pos) * move
        angle = np.arctan2(food_pos[1] - home_pos[1], food_pos[0] - home_pos[0])
    else:
        # Return
        move = (frame - half_frames) / half_frames
        head = food_pos + (home_pos - food_pos) * move
        angle = np.arctan2(home_pos[1] - food_pos[1], home_pos[0] - food_pos[0])

    t = frame * 0.02
    x_body, y_body = get_backbone_points(head, angle, t)
    line.set_data(x_body, y_body)
    head_marker.set_data([x_body[0]], [y_body[0]])
    return [line, head_marker]

plt.close(fig)
anim = animation.FuncAnimation(fig, animate, frames=n_frames, interval=20, blit=True)
HTML(anim.to_jshtml())

In [ ]:
# Download the GIF of the visualization
anim.save('mandarinfish_backbone_home-food.gif', writer='pillow', fps=30)

files.download('mandarinfish_backbone_home-food.gif')

Note: In this simulation, the fish's backbone is generated from the head forward along its direction of travel.

As a result, the segments of the body are projected "in front" of the head rather than trailing behind it.

This is a common convention in robotics and biomimetic simulations because it simplifies the mathematics and keeps the sinusoidal locomotion robust.

If you look into scientific publications and open-source demos, you'll find many follow this approach—often mentioning that a more biologically accurate body (with segments trailing directly behind the head) would require more complex coordinate transformations.

For visualization and analysis, this standard method is widely accepted and makes it easy to animate both movement and wave propagation, with the head always serving as the anchor point.

---


To make the simulation more realistic and challenging, we introduce obstacle avoidance—one of the key problems Mandarinfish face in their natural habitat.

Coral reefs act as both shelter and barriers, so it's essential for the fish to navigate efficiently around them.

In this part, I focus on implementing a basic avoidance algorithm, where the fish detects the coral as it approaches and intelligently adjusts its trajectory to reach the food target safely.


In [ ]:
# Coral obstacle placement (simple circle)
coral_center = np.array([10, 5]) # Typical position between home and food
coral_radius = 0.5 # Reasonable coral size

In [ ]:
# Main simulation figure
fig, ax = plt.subplots(figsize=(8, 3))
[line] = ax.plot([], [], '-o')
[head_marker] = ax.plot([], [], 'o', color='#FFB347', markersize=12, label="Head") # Mandarinfish's head visualization
ax.set_xlim(-2, 25)
ax.set_ylim(-2, 10)
ax.scatter([home_pos[0]], [home_pos[1]], c="g", label="Home", s=50)
ax.scatter([food_pos[0]], [food_pos[1]], c="r", label="Food", s=50)
ax.legend()
plt.title("mandarinfish_obstacle_1")
plt.tight_layout()

# Add the coral
coral_circle = plt.Circle(coral_center, coral_radius, color='#5EAB9E', alpha=0.4, label='Coral Reef')
ax.add_patch(coral_circle)

def is_near_coral(head_pos, coral_center, coral_radius):
    return np.linalg.norm(head_pos - coral_center) < (coral_radius + body_length*0.3)

# Variables to keep state for avoidance phase
in_avoidance = False
arc_phase = 0

def escapes_coral(head, food, coral_center, coral_radius):
    # Returns True if the direct path from head to food does not cross coral
    # Source: math for circle-line intersection
    # Vector from head to food
    d = food - head
    f = head - coral_center
    a = np.dot(d, d)
    b = 2 * np.dot(f, d)
    c = np.dot(f, f) - coral_radius**2

    discriminant = b**2 - 4*a*c
    return discriminant < 0  # True if no intersection

def animate(frame):
    global in_avoidance, arc_phase
    # Calculate progress along trajectory
    if frame < half_frames:
        move = frame / half_frames
        head = home_pos + (food_pos - home_pos) * move
    else:
        move = (frame - half_frames) / half_frames
        head = food_pos + (home_pos - food_pos) * move

    # Now decide orientation: avoidance or classic

    if is_near_coral(head, coral_center, coral_radius):
        if not in_avoidance:
            # Initial entry in avoidance
            arc_phase = np.arctan2(head[1] - coral_center[1], head[0] - coral_center[0])
            in_avoidance = True
        # Move the head along the arc
        arc_phase += 0.03 # increment sens of rotation, can be negative for other direction
        avoid_radius = coral_radius + body_length * 0.3
        head = coral_center + avoid_radius * np.array([np.cos(arc_phase), np.sin(arc_phase)])
        angle = arc_phase + np.pi/2 # tangent to the arc
        if escapes_coral(head, food_pos, coral_center, coral_radius):
            in_avoidance = False
    else:
        # Normal movement toward food/home
        if frame < half_frames:
            angle = np.arctan2(food_pos[1] - head[1], food_pos[0] - head[0])
        else:
            angle = np.arctan2(home_pos[1] - head[1], home_pos[0] - head[0])

    # Generate body points with final angle
    t = frame * 0.02
    x_body, y_body = get_backbone_points(head, angle, t)
    line.set_data(x_body, y_body)
    head_marker.set_data([x_body[0]], [y_body[0]])
    return [line, head_marker]


plt.close(fig)
anim = animation.FuncAnimation(fig, animate, frames=n_frames, interval=20, blit=True)
HTML(anim.to_jshtml())

In [ ]:
# Download the GIF of the visualization
anim.save('mandarinfish_obstacle_1.gif', writer='pillow', fps=30)

files.download('mandarinfish_obstacle_1.gif')

Building on the initial avoidance algorithm, I now add more coral obstacles.

This tests the robustness and flexibility of the fish's path-planning: the simulation demonstrates how the fish adapts its route to reach its goal despite increasingly complex barriers.


In [ ]:
# Many corals

coral_centers = [np.array([45, 42]), np.array([20, 17]), np.array([63, 63])]
coral_radiuss   = [1.8, 2.5, 3.0]


# Trajectory home-food-home
start_pos = np.array([0.0, 0.0])      # Starting position center
home_pos = np.array([0.0, 0.0])       # "Home” location (same as starting position)
food_pos = np.array([72.0, 69.0])     # Food target location

# Need to increase n_frame (bigger)
# Cut out the path
n_frames = 500
half_frames = n_frames // 2

# Direction
direction = (food_pos - home_pos) / np.linalg.norm(food_pos - home_pos)  # Unit vector
step_size = np.linalg.norm(food_pos - home_pos) / half_frames # Step size per frame

In [ ]:
# Main simulation figure
fig, ax = plt.subplots(figsize=(8, 8))
[line] = ax.plot([], [], '-o')
[head_marker] = ax.plot([], [], 'o', color='#FFB347', markersize=12, label="Head") # Mandarinfish's head visualization
ax.set_xlim(-2, 90)
ax.set_ylim(-2, 90)
ax.scatter([home_pos[0]], [home_pos[1]], c="g", label="Home", s=50)
ax.scatter([food_pos[0]], [food_pos[1]], c="r", label="Food", s=50)
ax.legend()
plt.title("mandarinfish_obstacles_3")
plt.tight_layout()

# For multiple coral obstacles
for center, radius in zip(coral_centers, coral_radiuss):
    coral_circle = plt.Circle(center, radius, color='#5EAB9E', alpha=0.4)
    ax.add_patch(coral_circle)

def is_near_any_coral(head_pos, coral_centers, coral_radiuss):
    for center, radius in zip(coral_centers, coral_radiuss):
        if np.linalg.norm(head_pos - center) < (radius + body_length * 0.3):
            return True, center, radius
    return False, None, None

# Variables to keep state for avoidance phase
in_avoidance = False
arc_phase = 0
arc_sense = 0.03

def compute_arc_direction(head, food, coral_center):
    vec_head = head - coral_center
    vec_food = food - coral_center
    cross = np.cross(vec_head, vec_food)
    # If cross-product is positive anti clockwise / otherwise clockwise
    return 0.03 if cross > 0 else -0.03

def escapes_coral(head, food, coral_center, coral_radius):
    # Returns True if the direct path from head to food does not cross coral
    # Source: math for circle-line intersection
    # Vector from head to food
    d = food - head
    f = head - coral_center
    a = np.dot(d, d)
    b = 2 * np.dot(f, d)
    c = np.dot(f, f) - coral_radius**2

    discriminant = b**2 - 4*a*c
    return discriminant < 0  # True if no intersection

def animate(frame):
    global in_avoidance, arc_phase, arc_sense
    # Calculate progress along trajectory
    if frame < half_frames:
        move = frame / half_frames
        head = home_pos + (food_pos - home_pos) * move
    else:
        move = (frame - half_frames) / half_frames
        head = food_pos + (home_pos - food_pos) * move

    # Now decide orientation: avoidance or classic
    near, coral_center, coral_radius = is_near_any_coral(head, coral_centers, coral_radiuss)
    if near:
        if not in_avoidance:
            arc_phase = np.arctan2(head[1] - coral_center[1], head[0] - coral_center[0])
            arc_sense = compute_arc_direction(head, food_pos, coral_center)
            in_avoidance = True
        arc_phase += arc_sense
        avoid_radius = coral_radius + body_length * 0.3
        head = coral_center + avoid_radius * np.array([np.cos(arc_phase), np.sin(arc_phase)])
        angle = arc_phase + np.pi/2
        if escapes_coral(head, food_pos, coral_center, coral_radius):
            in_avoidance = False
    else:
        # Normal movement toward food/home
        if frame < half_frames:
            angle = np.arctan2(food_pos[1] - head[1], food_pos[0] - head[0])
        else:
            angle = np.arctan2(home_pos[1] - head[1], home_pos[0] - head[0])


    # Generate body points with final angle
    t = frame * 0.02
    x_body, y_body = get_backbone_points(head, angle, t)
    line.set_data(x_body, y_body)
    head_marker.set_data([x_body[0]], [y_body[0]])
    return [line, head_marker]


plt.close(fig)
anim = animation.FuncAnimation(fig, animate, frames=n_frames, interval=20, blit=True)
HTML(anim.to_jshtml())

In [ ]:
# Download the GIF of the visualization
anim.save('mandarinfish_obstacles_3.gif', writer='pillow', fps=30)

files.download('mandarinfish_obstacles_3.gif')

In this section, the fish avoids each coral by following the optimal shortest path:

It stays on a circular arc until the direct line to the target no longer crosses the obstacle.

This strategy is called "tangent as soon as possible" and is standard in mobile robotics and path planning for minimal travel distance.

The result is technically optimal, but some avoidance arcs may appear visually brief or incomplete depending on geometry.

---

Now, I enhance the avoidance algorithm by adding a minimum arc condition:

The fish must follow at least a quarter or half-circular path (e.g., at least π/2 or π radians) around each coral before switching back to its direct path.

This adjustment guarantees smoother and more visually expressive obstacle avoidance, especially for demonstrations and educational purposes.


In [ ]:
# Variables to keep state for avoidance phase and arc tracking
in_avoidance = False
arc_phase = 0
arc_sense = 0.03
arc_phase_entry = 0
min_arc = np.pi        # Minimum angle in radians for avoidance (π for half circle, π/2 for quarter)
current_coral_center = None
current_coral_radius = None

In [ ]:
# Name of the plot
ax.set_title("mandarinfish_circular_avoidance")

def animate(frame):
    global in_avoidance, arc_phase, arc_sense, arc_phase_entry, current_coral_center, current_coral_radius
    # Calculate progress along the home-food-home trajectory
    if frame < half_frames:
        move = frame / half_frames
        head = home_pos + (food_pos - home_pos) * move
    else:
        move = (frame - half_frames) / half_frames
        head = food_pos + (home_pos - food_pos) * move

    if in_avoidance:
        # Continue following current coral arc until both conditions are met (geometry and minimum arc length)
        arc_phase += arc_sense
        avoid_radius = current_coral_radius + body_length * 0.3
        head = current_coral_center + avoid_radius * np.array([np.cos(arc_phase), np.sin(arc_phase)])
        angle = arc_phase + np.pi/2
        angle_traversed = np.abs(arc_phase - arc_phase_entry)
        # Exit avoidance only after the fish has traveled at least min_arc and the direct path is clear
        if escapes_coral(head, food_pos, current_coral_center, current_coral_radius) and angle_traversed > min_arc:
            in_avoidance = False
            current_coral_center = None
            current_coral_radius = None
    else:
        # Enter avoidance mode for the first nearby coral
        near, coral_center, coral_radius = is_near_any_coral(head, coral_centers, coral_radiuss)
        if near:
            arc_phase = np.arctan2(head[1] - coral_center[1], head[0] - coral_center[0])
            arc_phase_entry = arc_phase
            arc_sense = compute_arc_direction(head, food_pos, coral_center)
            in_avoidance = True
            current_coral_center = coral_center
            current_coral_radius = coral_radius
            # Move forward right on the arc
            arc_phase += arc_sense
            avoid_radius = current_coral_radius + body_length * 0.3
            head = current_coral_center + avoid_radius * np.array([np.cos(arc_phase), np.sin(arc_phase)])
            angle = arc_phase + np.pi/2
        else:
            # Default swimming (straight path to goal)
            if frame < half_frames:
                angle = np.arctan2(food_pos[1] - head[1], food_pos[0] - head[0])
            else:
                angle = np.arctan2(home_pos[1] - head[1], home_pos[0] - head[0])

    # Compute backbone points and update visualization
    t = frame * 0.02
    x_body, y_body = get_backbone_points(head, angle, t)
    line.set_data(x_body, y_body)
    head_marker.set_data([x_body[0]], [y_body[0]])
    return [line, head_marker]

plt.close(fig)
anim = animation.FuncAnimation(fig, animate, frames=n_frames, interval=20, blit=True)
HTML(anim.to_jshtml())

In [ ]:
# Download the GIF of the visualization
anim.save('mandarinfish_circular_avoidance.gif', writer='pillow', fps=30)

files.download('mandarinfish_circular_avoidance.gif')

The Mandarin fish doesn't reach the food and go straight to the corals without following the main path.

In [ ]:
def is_close_to_food(head, food, threshold=2.0):
    # Returns True if the fish's head is sufficiently close to the food (goal)
    return np.linalg.norm(head - food) < threshold

last_avoided_center = None  # Memory for last coral avoided
avoidance_recovery = 4  # minimum allowed distance from last avoided coral before new avoidance


In [ ]:
# Name of the plot
ax.set_title("mandarinfish_circular_avoidance_food")

def animate(frame):
    global in_avoidance, arc_phase, arc_sense, arc_phase_entry, current_coral_center, current_coral_radius
    # Calculate progress along the home-food-home trajectory
    if frame < half_frames:
        move = frame / half_frames
        head = home_pos + (food_pos - home_pos) * move
        is_arrival = is_close_to_food(head, food_pos, threshold=2.0)
    else:
        move = (frame - half_frames) / half_frames
        head = food_pos + (home_pos - food_pos) * move
        is_arrival = is_close_to_food(head, home_pos, threshold=2.0)

    # STOP avoidance if close to current goal (food or home)
    if is_arrival:
        in_avoidance = False
        current_coral_center = None
        current_coral_radius = None
        if frame < half_frames:
            angle = np.arctan2(food_pos[1] - head[1], food_pos[0] - head[0])
        else:
            angle = np.arctan2(home_pos[1] - head[1], home_pos[0] - head[0])
    else:
        if in_avoidance:
            arc_phase += arc_sense
            avoid_radius = current_coral_radius + body_length * 0.3
            head = current_coral_center + avoid_radius * np.array([np.cos(arc_phase), np.sin(arc_phase)])
            angle = arc_phase + np.pi/2
            angle_traversed = np.abs(arc_phase - arc_phase_entry)
            # Check direct path to correct goal
            if frame < half_frames:
                path_clear = escapes_coral(head, food_pos, current_coral_center, current_coral_radius)
            else:
                path_clear = escapes_coral(head, home_pos, current_coral_center, current_coral_radius)

            if path_clear and angle_traversed > min_arc:
                in_avoidance = False
                current_coral_center = None
                current_coral_radius = None
        else:
            near, coral_center, coral_radius = is_near_any_coral(head, coral_centers, coral_radiuss)
            if near:
                arc_phase = np.arctan2(head[1] - coral_center[1], head[0] - coral_center[0])
                arc_phase_entry = arc_phase
                if frame < half_frames:
                    arc_sense = compute_arc_direction(head, food_pos, coral_center)
                else:
                    arc_sense = compute_arc_direction(head, home_pos, coral_center)
                in_avoidance = True
                current_coral_center = coral_center
                current_coral_radius = coral_radius
                arc_phase += arc_sense
                avoid_radius = current_coral_radius + body_length * 0.3
                head = current_coral_center + avoid_radius * np.array([np.cos(arc_phase), np.sin(arc_phase)])
                angle = arc_phase + np.pi/2
            else:
                # Straight toward correct goal
                if frame < half_frames:
                    angle = np.arctan2(food_pos[1] - head[1], food_pos[0] - head[0])
                else:
                    angle = np.arctan2(home_pos[1] - head[1], home_pos[0] - head[0])

    # Compute backbone points and update visualization
    t = frame * 0.02
    x_body, y_body = get_backbone_points(head, angle, t)
    line.set_data(x_body, y_body)
    head_marker.set_data([x_body[0]], [y_body[0]])
    return [line, head_marker]

plt.close(fig)
anim = animation.FuncAnimation(fig, animate, frames=n_frames, interval=20, blit=True)
HTML(anim.to_jshtml())

In [ ]:
# Download the GIF of the visualization
anim.save('mandarinfish_circular_avoidance_food.gif', writer='pillow', fps=30)

files.download('mandarinfish_circular_avoidance_food.gif')

Now, the Mandarinfish navigates toward the food source but may still deviate from the main path, even when a more direct route appears possible. According to the literature in biomechanics and robotics, this is a well-known phenomenon:
the shape and placement of obstacles can force a detour, or even create situations where the simulated fish "prefers" to stay on an avoidance arc rather than return immediately to the optimal trajectory.

Such outcomes arise especially when obstacles are large or positioned near critical points on the path—sometimes leading to visually suboptimal but biomechanically credible movement.

As a final step, I enhance the simulation aesthetically: adding colors, shapes, and environmental details to better illustrate the Mandarinfish in its natural setting.

This both improves readability and helps make the project visually compelling.

---

To do this improvement, we will us the first three corals simulation: when the Mandarinfish avoid all the corals and follow mostly correctly the main path.

In [ ]:
# Marine background and axes
fig, ax = plt.subplots(figsize=(8, 8))
ax.set_facecolor('#BEE6E8')
ax.set_xlim(-2, 90)
ax.set_ylim(-2, 90)
plt.title("Mandarinfish Biomimetic Navigation - Coral Reef", fontsize=15)
plt.tight_layout()

# Custom coral colors - HEX uniform
coral_colors = ['#CCA2DB', '#E09058', '#40C2A3', '#FFD700', '#B22222']
for i, (center, radius) in enumerate(zip(coral_centers, coral_radiuss)):
    coral_circle = plt.Circle(center, radius, color=coral_colors[i % len(coral_colors)], alpha=0.5, ec='#A0522D', lw=2, zorder=2)
    ax.add_patch(coral_circle)

# Home and Food
ax.scatter([home_pos[0]], [home_pos[1]], c="#576926", label="Home", s=120, marker="h", edgecolors="k", zorder=5)
ax.scatter([food_pos[0]], [food_pos[1]], c="#FFD700", label="Food", s=120, marker="*", edgecolors="k", zorder=5)
ax.legend(loc="upper left")

# Mandarinfish rainbow palette
segment_colors = ['#2DBDC4', '#DB8B14', '#1C6DA3', '#F0A007']

# No main line: only segments drawn
[head_marker] = ax.plot([], [], 'o', color="#FFB347", markersize=12, label="Head")
segment_lines = []  # List that will keep the segment Line2D for each frame

# State for avoidance
in_avoidance = False
arc_phase = 0
arc_sense = 0.03

def is_near_any_coral(head_pos, coral_centers, coral_radiuss):
    for center, radius in zip(coral_centers, coral_radiuss):
        if np.linalg.norm(head_pos - center) < (radius + body_length * 0.3):
            return True, center, radius
    return False, None, None

def compute_arc_direction(head, food, coral_center):
    vec_head = head - coral_center
    vec_food = food - coral_center
    cross = np.cross(vec_head, vec_food)
    return 0.03 if cross > 0 else -0.03

def escapes_coral(head, food, coral_center, coral_radius):
    d = food - head
    f = head - coral_center
    a = np.dot(d, d)
    b = 2 * np.dot(f, d)
    c = np.dot(f, f) - coral_radius**2
    discriminant = b**2 - 4*a*c
    return discriminant < 0

# Animate with colored segments
def animate(frame):
    global in_avoidance, arc_phase, arc_sense, segment_lines

    # Remove previous segment lines
    for seg in segment_lines:
        seg.remove()
    segment_lines = []

    if frame < half_frames:
        move = frame / half_frames
        head = home_pos + (food_pos - home_pos) * move
    else:
        move = (frame - half_frames) / half_frames
        head = food_pos + (home_pos - food_pos) * move

    near, coral_center, coral_radius = is_near_any_coral(head, coral_centers, coral_radiuss)
    if near:
        if not in_avoidance:
            arc_phase = np.arctan2(head[1] - coral_center[1], head[0] - coral_center[0])
            arc_sense = compute_arc_direction(head, food_pos, coral_center)
            in_avoidance = True
        arc_phase += arc_sense
        avoid_radius = coral_radius + body_length * 0.7
        head = coral_center + avoid_radius * np.array([np.cos(arc_phase), np.sin(arc_phase)])
        angle = arc_phase + np.pi/2
        if escapes_coral(head, food_pos, coral_center, coral_radius):
            in_avoidance = False
    else:
        if frame < half_frames:
            angle = np.arctan2(food_pos[1] - head[1], food_pos[0] - head[0])
        else:
            angle = np.arctan2(home_pos[1] - head[1], home_pos[0] - head[0])

    t = frame * 0.02
    x_body, y_body = get_backbone_points(head, angle, t)

    for i in range(len(x_body)-1):
        [seg] = ax.plot(x_body[i:i+2], y_body[i:i+2], color=segment_colors[i % len(segment_colors)],
                       lw=7, solid_capstyle='round', zorder=6)
        segment_lines.append(seg)
    head_marker.set_data([x_body[0]], [y_body[0]])


    return segment_lines + [head_marker]

plt.close(fig)
anim = animation.FuncAnimation(fig, animate, frames=n_frames, interval=20, blit=True)
HTML(anim.to_jshtml())


In [ ]:
# Download the GIF of the visualization
anim.save('mandarinfish_final_simulation.gif', writer='pillow', fps=30)

files.download('mandarinfish_final_simulation.gif')

This final simulation brings together all the essential elements:
a Mandarinfish with a realistic undulating backbone, navigating intelligently through a colorful coral environment to reach its food, all while dynamically avoiding obstacles.

Along the way, some minor issues typical of biomimetic models—such as occasional detours and non-optimal paths due to obstacle placement—were discussed, reflecting real complexities in natural and robotic navigation.